In [10]:
from anngel.datasets.openbiolink import OpenBioLinkDataset
import pandas as pd

OUT_FILE = "./gene_df.csv"
EMAIL = "ENTER YOUR EMAIL HERE"


ds = OpenBioLinkDataset("../tmp/openbiolink/")
nodes_df = ds.get_nodes().query('node_id.str.startswith("NCBIGENE:")')
gene_ids = [s.replace("NCBIGENE:", "") for s in nodes_df["node_id"]]

from Bio import Entrez

Entrez.email = EMAIL


def get_gene_info(gene_ids):
    handle = Entrez.esummary(db="gene", id=",".join(gene_ids))
    result = Entrez.read(handle)
    handle.close()

    return result["DocumentSummarySet"]["DocumentSummary"]


from tqdm import tqdm

batch_size = 1000
gene_infos = []
for i in tqdm(range(0, len(gene_ids), batch_size)):
    gene_infos.extend(get_gene_info(gene_ids[i : i + batch_size]))

gene_names = [x["Name"] for x in gene_infos]
gene_df = pd.DataFrame(
    {
        "gene_name": gene_names,
        "gene_label": gene_ids,
        "gene_id": nodes_df.index,
    }
)
gene_df.to_csv(OUT_FILE, index=False)

100%|██████████| 20/20 [00:57<00:00,  2.86s/it]
